In [1]:
from deepface import DeepFace

In [3]:
result = DeepFace.find(img_path = "./tests/im2.jpg", db_path = "./db_rostos")

Finding representations: 100%|██████████| 2/2 [00:03<00:00,  1.92s/it]

23-12-16 21:33:37 - Representations stored in ./db_rostos/representations_vgg_face.pkl file.Please delete this file when you add new identities in your database.
23-12-16 21:33:37 - find function lasts 4.022372007369995 seconds


É necessário excluir o arquivo representations_vgg_face.pkl quando adicionar uma pessoa na base de dados!

In [9]:
from ultralytics import YOLO
model = YOLO('runs/detect/train17/weights/best.pt')
results = model.predict('lacto.jpg')
box = results[0].boxes.xyxy.tolist()


image 1/1 c:\Users\Eduardo\Documents\FaceRecognition_FeatureExtractor\lacto.jpg: 640x480 4 Faces, 120.8ms
Speed: 5.5ms preprocess, 120.8ms inference, 11.0ms postprocess per image at shape (1, 3, 640, 480)


In [15]:
import cv2
import numpy as np
# image = cv2.imread("lacto.jpg")
def process_image(image):
    for face in box:
        try:
            # Pontos XY
            x0 = int(face[0])
            y0 = int(face[1])
            x1 = int(face[2])
            y1 = int(face[3])

            # Recorte do rosto
            individual_face = image[y0:y1, x0:x1]

            # Inicio e Fim do rosto
            start_point = (x0, y0)
            end_point = (x1, y1)

            # Inicio e fim dos textos
            start_space = (x0, y1)
            end_space = (x1, y1+100)
            
            # Cor e espessura do retrato
            color = (172, 52, 52)
            thickness = 2

            # Dados sobre o rosto
            person = DeepFace.find(individual_face, db_path = "./db_rostos")
            per = 'N/A' if len(person[0]) == 0 else person[0].identity[0].split('/')[2].split('.')[0]
            res = DeepFace.analyze(individual_face)[0]

            # Retrato na foto original e espaco para o texto
            image = cv2.rectangle(image, start_point, end_point, color, thickness)
            image = cv2.rectangle(image, start_space, end_space, color, -1) 

            # # Resultados do rosto encontrado
            image = cv2.putText(image, f'Person: {per}', (x0+5, y1+18), cv2.FONT_HERSHEY_SIMPLEX,  
                        0.7, (255, 255, 255), 1) 
            image = cv2.putText(image, f'Gender: {res["dominant_gender"]}', (x0+5, y1+38), cv2.FONT_HERSHEY_SIMPLEX,  
                        0.7, (255, 255, 255), 1) 
            image = cv2.putText(image, f'Age: {res["age"]}', (x0+5, y1+58), cv2.FONT_HERSHEY_SIMPLEX,  
                        0.7, (255, 255, 255), 1) 
            image = cv2.putText(image, f'Emotion: {res["dominant_emotion"]}', (x0+5, y1+78), cv2.FONT_HERSHEY_SIMPLEX,  
                        0.7, (255, 255, 255), 1) 
            image = cv2.putText(image, f'Race: {res["dominant_race"]}', (x0+5, y1+98), cv2.FONT_HERSHEY_SIMPLEX,  
                        0.7, (255, 255, 255), 1) 
            
        except:
            pass
    return image
    

# Plataforma Web para uso do conhecimento

In [ ]:
import dash
import numpy as np
import dash_html_components as html
from dash_canvas import DashCanvas
from dash import dcc
from dash.dependencies import Input, Output, State
from dash_canvas.utils import array_to_data_url, image_string_to_PILImage

app = dash.Dash(__name__)

canvas_width = 650

app.layout = html.Div([
    html.H2('Reconhecimento facial e análise de características', style={'text-align':'center'}),


    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Arraste e solte ou ',
            html.A('selecione uma imagem para analisar')
        ]),
        style={
            'width': '99%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-image-upload',style={'width': '100%', 'display': 'flex', 'align-items':'center', 'justify-content':'center'})
])


def parse_contents(contents, filename, date):
    img = image_string_to_PILImage(contents)
    img.save('img.jpg')
    img = cv2.imread("img.jpg")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = process_image(img)
    pix = np.array(img)
    print(pix)
    img_content = array_to_data_url(pix)
    return html.Div([DashCanvas(id='canvaas_image',
                                image_content=img_content,
                                lineWidth=5,
                                lineColor='red',
                                width=canvas_width)])


@app.callback(Output('output-image-upload', 'children'),
              Input('upload-image', 'contents'),
              State('upload-image', 'filename'),
              State('upload-image', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


if __name__ == '__main__':
    app.run_server(debug=True)

23-12-16 21:41:36 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:41:36 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 21:41:36 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:41:36 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 21:41:36 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:41:36 - There are 2 representations fou

Action: race: 100%|██████████| 4/4 [00:00<00:00,  8.14it/s]  


23-12-16 21:41:45 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:41:45 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 21:41:45 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:41:45 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 21:41:45 - find function lasts 0.17990875244140625 seconds


Action: race: 100%|██████████| 4/4 [00:00<00:00,  8.04it/s]  


23-12-16 21:41:46 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:41:46 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 21:41:46 - find function lasts 0.16997909545898438 seconds


...

23-12-16 21:43:01 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:43:01 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 21:43:01 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:43:01 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 21:43:01 - find function lasts 0.17154502868652344 seconds


Action: race: 100%|██████████| 4/4 [00:00<00:00,  7.84it/s]  


23-12-16 21:43:02 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-16 21:43:02 - There are 2 representations found in representations_vgg_face.pkl
23-12-16 21:43:02 - find function lasts 0.17554688453674316 seconds


Action: race: 100%|██████████| 4/4 [00:00<00:00,  7.56it/s]  


[[[ 69  69  61]
  [ 70  70  62]
  [ 71  71  63]
  ...
  [ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]]

 [[ 69  69  61]
  [ 70  70  62]
  [ 71  71  63]
  ...
  [ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]]

 [[ 69  69  61]
  [ 69  69  61]
  [ 71  71  63]
  ...
  [ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]]

 ...

 [[ 75  76  60]
  [ 73  74  58]
  [ 79  80  64]
  ...
  [119 105 102]
  [129 115 112]
  [137 123 120]]

 [[ 72  70  55]
  [ 70  68  53]
  [ 78  76  61]
  ...
  [130 116 113]
  [148 134 131]
  [160 146 143]]

 [[ 70  68  53]
  [ 67  65  50]
  [ 71  69  54]
  ...
  [137 123 120]
  [159 145 142]
  [175 161 158]]]
